In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 

import torch

import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision.transforms as transforms
from torch.utils.data import  Dataset, TensorDataset, DataLoader


import lib.pytorch_trainer as ptt

from src.imgnet_utils import denormalize

from src.data_loader import _create_dataLoader

from src.Dataset import KaggleSafeDriverDataset

from src.plot_utils import (plot_classes, plot_distribution,
                            statistical_analysis_image, classDistribution,
                            plot_metrics,visualize_predictions,
                            plot_cm_train_valid,plot_layers_weight)
      
from src.convnet_models import (MyResNet, MyInception, MyDenseNet,MyVgg16Net)
from src.extractor_utils import (predict, getPrediction,save_prediction,
                                 load_prediction, torch_summarize,RandomSearch)

from utils.utils import (create_submission ,metrics2csv)

# Making results reproducible

In [3]:
torch.backends.cudnn.deterministic = True
seed=1719
np.random.seed(seed)
torch.manual_seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed) 

# Checking for resources avaliables

Checking if the GPU is free. 

In [4]:
!free -h
!nvidia-smi

              total        used        free      shared  buff/cache   available
Mem:            29G        632M         22G        9.2M        6.3G         28G
Swap:            0B          0B          0B
Sun Feb  4 15:49:12 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.12                 Driver Version: 390.12                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   28C    P8    28W / 149W |     15MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                            

In [5]:
print("{} GPU's available:".format(torch.cuda.device_count()) )
cpu_count = torch.multiprocessing.cpu_count()
print("\ncpu_count: {}".format(cpu_count))

1 GPU's available:

cpu_count: 8


In [6]:
use_gpu = True
use_DataParalel= False
use_CPU= False       

if use_gpu:
    if use_DataParalel: 
        print("Using DataParalel in all {} GPUS".format(torch.cuda.device_count()))
    else:
        print('Using only one GPU') 
if use_CPU:         # http://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html 
    print("Using {} CPU's".format(cpu_count))

Using only one GPU



# Path to features

In [7]:
path2features= "./features/" 

# RandomSearch on hyperparameters

In [8]:
# Flag to optimze hyperparameters
hiper_tunning = True

# Choosing Model

In [9]:
use_resnet = False
use_inception = False
use_denseNet = True
use_vgg16 = False
if use_resnet:
    print('Using ResNet model')
    model_name = "ResNet"
    model = MyResNet()
elif use_inception:
    print('Using Inception model')
    model_name = "Inception"
    model = MyInception()
elif use_denseNet:
    print('Using DenseNet model')
    model_name = "DenseNet"    
    model = MyDenseNet() 
elif use_vgg16:
    print('Using VGG16 model')
    model_name = "vgg16"    
    model = MyVgg16Net()

Using DenseNet model


In [10]:
if use_gpu:
    if use_DataParalel:
        print("Using all GPU's ")
        model.mrnc = torch.nn.DataParallel(model.mrnc) #device_ids=[1,3]
        model.mrnc = model.mrnc.cuda()
        model.mrnd = torch.nn.DataParallel(model.mrnd) #device_ids=[1,3]
        model.mrnd = model.mrnd.cuda()
    else:
        print('Using GPU')# {}'.format(device_id))
        model.cuda()
else:
    print("Using CPU's")

Using GPU


# Loading features

In [11]:
model_name
print(model.mrnd)

MyDenseNetDens(
  (dens1): Linear(in_features=2208, out_features=512)
  (dens2): Linear(in_features=512, out_features=128)
  (dens3): Linear(in_features=128, out_features=10)
)


### Load the features which were already extracted from the image using the model. 
* This has been run with the first (n-1) layers of the network. The last layer is our classifier

In [11]:
load_feat= load_prediction(path2features,model_name)

Loaded features with shapes: 


train:
pred torch.Size([17940, 1000]), true torch.Size([17940])

valid:
pred torch.Size([4484, 1000]), true torch.Size([4484])

test:
pred torch.Size([79726, 1000]), true torch.Size([79726])


In [12]:
load_feat['train'][0].shape,load_feat['train'][1].shape

(torch.Size([17940, 1000]), torch.Size([17940]))

In [13]:
conv_dset ={
'train': TensorDataset(load_feat['train'][0], load_feat['train'][1]),
'valid': TensorDataset(load_feat['valid'][0], load_feat['valid'][1]),
'test': TensorDataset(load_feat['test'][0], load_feat['test'][1])
}

# Creating Dataloaders

In [14]:
batch_size= 32

dset_loaders_convnet = _create_dataLoader(conv_dset, batch_size, 
                        pin_memory=False, use_shuffle= True)

In [15]:
dset_convnet_sizes = {x: len(dset_loaders_convnet[x]) for x in ['train','valid', 'test']} 
dset_convnet_sizes

{'test': 2492, 'train': 561, 'valid': 141}

# Trainning Model

In [16]:
if hiper_tunning: 
    model_name += 'RandomSearch' 

path2saveModel = './models/'+model_name+'DistractDriver' 

savebest = ptt.ModelCheckpoint(path2saveModel,reset=True, verbose=0)

In [21]:
num_epochs = 50
loss_fn = nn.CrossEntropyLoss()
params = filter(lambda p: p.requires_grad, model.parameters())
optimizer =  optim.Adam(params, lr=1e-3,weight_decay=0)

params = {'model' : model.mrnd, 
    'criterion': loss_fn,  
    'optimizer': optimizer, 
    'callbacks': [savebest, ptt.AccuracyMetric()] #ptt.PlotCallback(),
}

In [22]:
# Search space for tunning hyperparameters 
args= {'lr':[1e-5,1e-2],
       'weight_decay':[1e-8,1e-3] }

In [23]:
if hiper_tunning: 

    output = RandomSearch(params,args,
                          num_epochs=num_epochs,path2saveModel=path2saveModel,
                          dset_loaders_convnet=dset_loaders_convnet,
                          MAX_IT=20,verbose=1)
    
    trainer = output['best_trainer']
    print('****************************')
    print('\nBest parameters {}'.format(output['best_parameters']))
    print('\nBest result {}'.format(output['best_result']))

else:
    params['callbacks'].append(ptt.PrintCallback())
    scheduler = StepLR(optimizer, step_size=5, gamma=0.55)
    trainer = ptt.DeepNetTrainer(use_gpu=use_gpu,lr_scheduler=scheduler,**params)
    trainer.fit_loader(num_epochs, dset_loaders_convnet['train'], dset_loaders_convnet['valid'])

Iteration 0/20


RuntimeError: size mismatch at /pytorch/torch/lib/THC/generic/THCTensorMathBlas.cu:243

In [ ]:
print(torch_summarize(model))

# Loading best model

In [ ]:
trainer.load_state(path2saveModel)
model.mrnd = trainer.model

In [ ]:
metrics2csv(trainer, model_name)

In [ ]:
train_eval = trainer.evaluate_loader(dset_loaders_convnet['train'])
valid_eval = trainer.evaluate_loader(dset_loaders_convnet['valid'])

In [ ]:
train_eval, valid_eval

# Dataset without shuffling 

#### Utilized to plot mismatch cases

In [ ]:
path2train = "/home/Prashanth/dl4cv-project/data/train"  

In [ ]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std  = np.array([0.229, 0.224, 0.225])

img_width = img_height=224 #to use InceptionV3 it must img_width and img_height be changed to 300

# Data augmentation and normalization for training 
data_transforms = {
    'valid': transforms.Compose([
        transforms.Scale((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean, imagenet_std),
    ]),
}   

In [ ]:
batch_size = 32
use_only = 1.0 # Use only is the percentage of the full dataset

In [ ]:
dsets = {
    'valid': KaggleSafeDriverDataset(path2train,
                                     transforms=data_transforms['valid'],
                                     use_only=use_only, is_val=True, val_size=0.2),
}

In [ ]:

dset_loaders_wshuffle = _create_dataLoader(dsets, batch_size,
                                           pin_memory=False, use_shuffle= False)

# Evaluating resultings 

In [ ]:
#we need to use all dloader, because this one has use_shuffle false
result_train = predict(dset_loaders_convnet['train'], model.mrnd, use_gpu=use_gpu)

result_valid = predict(dset_loaders_wshuffle['valid'], model, use_gpu=use_gpu)

result_test = predict(dset_loaders_convnet['test'], model.mrnd, use_gpu=use_gpu)

In [ ]:
predictions_out = {'train': (result_train['pred'], result_train['true'], model_name),
                   'valid': (result_valid['pred'], result_valid['true'], model_name),
                   'test': (result_test['pred'], result_test['true'], model_name)}

In [ ]:
path2results = './results/'

In [ ]:
save_prediction(path2results,predictions_out)

In [ ]:
result_train = getPrediction(result_train)
result_valid = getPrediction(result_valid)
# result_test['pred'] must be an array of probabilities to make the submission

In [ ]:
correct_train = (result_train['true'] == result_train['pred']).sum()
correct_valid = (result_valid['true'] == result_valid['pred']).sum()

In [ ]:
print('Train: ', correct_train, '/', '17940' )# len(dsets['train'])
print('Valid: ', correct_valid, '/', '4484' ) # len(dsets['valid'])

# Make submission of the Test set

In [ ]:
import pandas as pd
import datetime
import os
from torch.nn.functional import softmax
from torch.autograd import Variable
predictions = softmax(Variable(result_test['pred'])).cpu().data.numpy().tolist()
predictions = np.around(predictions, decimals=3)
predictions = np.clip(predictions, 0.001, 0.999) 

test_id = result_test['true'].tolist()
for i in range(0, len(test_id)):
    test_id[i] = 'img_'+ str(test_id[i]) + '.jpg'

result_sample = pd.DataFrame(predictions, columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
result_sample.loc[:, 'img'] = pd.Series(test_id, index=result_sample.index)

cols = result_sample.columns.tolist()
cols = cols[-1:] + cols[:-1]

result_sample = result_sample[cols]
now = datetime.datetime.now()
if not os.path.isdir('subm'):
    os.mkdir('subm')
suffix = model_name + '_distracted_driver' + '_' + str(now.strftime("%Y-%m-%d-%H-%M"))
sub_file = os.path.join('subm', 'submission_' + suffix + '.csv')
result_sample.to_csv(sub_file, index=False)
print('done!')
#create_submission(result_test, model_name + '_distracted_driver')